##bert初探

###安裝transformers

In [0]:
%%bash
pip install transformers tqdm boto3 requests regex -q

In [0]:
import torch
from transformers import BertTokenizer # 載入中文 BERT 使用的 tokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.5.0+cu101


可以在 Hugging Face 團隊的 repo 裡看到所有可從 PyTorch Hub 載入的 BERT 預訓練模型。截至目前為止有以下模型可供使用：
*   bert-base-chinese
*   bert-base-uncased
*   bert-base-cased (英文)
*   bert-base-german-cased
*   bert-base-multilingual-uncased
*   bert-base-multilingual-cased
*   bert-large-cased
*   bert-large-uncased
*   bert-large-uncased-whole-word-masking
*   bert-large-cased-whole-word-masking

這些模型的參數都已經被訓練完成，而主要差別在於：


*   預訓練步驟時用的文本語言
*   有無分大小寫
*   層數的不同
*   預訓練時遮住 wordpieces 或是整個 word

###讓我們看看 tokenizer 裡頭的字典資訊

In [0]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


如上所示，中文 BERT 的字典大小約有 2.1 萬個 tokens。英文 BERT 的字典則大約是 3 萬 tokens 左右。我們可以瞧瞧中文 BERT 字典裡頭紀錄的一些 tokens 以及其對應的索引：

In [0]:
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, id))

token               index          
-------------------------
f2                   9675
28                   8143
鶘                    7871
##牧                 17345
00z                  9337
隻                    7407
##処                 14186
抨                    2846
##辈                 19834
rx                  12342


BERT 使用當初 Google NMT 提出的 WordPiece Tokenization ，將本來的 words 拆成更小粒度的 wordpieces，有效處理不在字典裡頭的詞彙 。中文的話大致上就像是 character-level tokenization，而有 ## 前綴的 tokens 即為 wordpieces。

以詞彙 fragment 來說，其可以被拆成 frag 與 ##ment 兩個 pieces，而一個 word 也可以獨自形成一個 wordpiece。wordpieces 可以由蒐集大量文本並找出其中常見的 pattern 取得。

另外有趣的是ㄅㄆㄇㄈ也有被收錄：

In [0]:
indices = list(range(647, 657))
some_pairs = [(t, idx) for t, idx in vocab.items() if idx in indices]
for pair in some_pairs:
    print(pair)

('ㄅ', 647)
('ㄆ', 648)
('ㄇ', 649)
('ㄉ', 650)
('ㄋ', 651)
('ㄌ', 652)
('ㄍ', 653)
('ㄎ', 654)
('ㄏ', 655)
('ㄒ', 656)


In [0]:
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text) # 將單詞轉成tokenize
ids = tokenizer.convert_tokens_to_ids(tokens) # 將單詞轉成字典裡tokenize對應的id

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')

[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761] ...


除了一般的 wordpieces 以外，BERT 裡頭有 5 個特殊 tokens 各司其職：



*   [CLS]：在做分類任務時其最後一層的 repr. 會被視為整個輸入序列的 repr.

*   [SEP]：有兩個句子的文本會被串接成一個輸入序列，並在兩句之間插入這個 token 以做區隔
*   [UNK]：沒出現在 BERT 字典裡頭的字會被這個 token 取代
*   [PAD]：zero padding 遮罩，將長度不一的輸入序列補齊方便做 batch 運算
*   [MASK]：未知遮罩，僅在預訓練階段會用到








如上例所示，[CLS] 一般會被放在輸入序列的最前面，而 zero padding 在之前的 Transformer 文章裡已經有非常詳細的介紹。[MASK] token 一般在 fine-tuning 或是 feature extraction 時不會用到，這邊只是為了展示預訓練階段的克漏字任務才使用的。

In [0]:
"""
這段程式碼載入已經訓練好的 masked 語言模型並對有 [MASK] 的句子做預測
"""
from transformers import BertForMaskedLM # import已經訓練好的 masked 語言模型

# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids])  # 創建tensor(1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor)  # 返回與input相同size的零矩陣 (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token 
maskedLM_model.eval()
with torch.no_grad(): #以達到暫時不追踪網絡參數中的導數的目的，總之是為了減少可能存在的計算和內存消耗
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
del maskedLM_model

# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')

輸入 tokens ： ['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
--------------------------------------------------
Top 1 (67%)：['[CLS]', '等', '到', '潮', '水', '來', '了', '，', '就', '知'] ...
Top 2 (25%)：['[CLS]', '等', '到', '潮', '水', '濕', '了', '，', '就', '知'] ...
Top 3 ( 2%)：['[CLS]', '等', '到', '潮', '水', '過', '了', '，', '就', '知'] ...


很明顯地，在這兩個情境裡頭「他」所代表的語義以及指稱的對象皆不同。如果仍使用沒蘊含上下文 / 語境資訊的詞向量，機器就會很難正確地「解讀」這兩個句子所蘊含的語義了。

現在讓我們跟隨這個 Colab 筆記本安裝 BERT 的視覺化工具 BertViz，看看 BERT 會怎麼處理這兩個情境：

In [0]:
# 安裝 BertViz
import sys
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']

# import packages
from transformers import BertTokenizer, BertModel
from bertviz import head_view

# 在 jupyter notebook 裡頭顯示 visualzation 的 helper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

clear_output()

以後就能非常輕鬆地將 BERT 內部的注意力機制視覺化出來：

In [0]:
model_version = 'bert-base-chinese' #使用中文 BERT
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)

# 情境 1 的句子
sentence_a = "胖虎叫大雄去買漫畫，"
sentence_b = "回來慢了就打他。"

# 得到 tokens 後丟入 BERT 取得 attention
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

# 交給 BertViz 視覺化
head_view(attention, tokens)

# 注意：執行這段程式碼以後只會顯示下圖左側的結果。

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

上面是 BERT 裡第 某 層 Encoder block 其中一個 head 的注意力結果。

圖中的線條代表該 head 在更新「他」（左側）的 repr. 時關注其他詞彙（右側）的注意力程度。越粗代表關注權重（attention weights）越高。很明顯地這個 head 具有一定的指代消解（Coreference Resolution）能力，能正確地關注「他」所指代的對象。

要處理指代消解需要對自然語言有不少理解，而 BERT 在沒有標注數據的情況下透過自注意力機制、深度雙向語言模型以及「閱讀」大量文本達到這樣的水準，是一件令人雀躍的事情。

當然 BERT 並不是第一個嘗試產生 contextual word repr. 的語言模型。在它之前最知名的例子有剛剛提到的 ELMo 以及 GPT：

ELMo 利用獨立訓練的雙向兩層 LSTM 做語言模型並將中間得到的隱狀態向量串接當作每個詞彙的 contextual word repr.；GPT 則是使用 Transformer 的 Decoder 來訓練一個中規中矩，從左到右的單向語言模型。你可以參考我另一篇文章：直觀理解 GPT-2 語言模型並生成金庸武俠小說來深入了解 GPT 與 GPT-2。

BERT 跟它們的差異在於利用 MLM（即克漏字）的概念及 Transformer Encoder 的架構，擺脫以往語言模型只能從單個方向（由左到右或由右到左）估計下個詞彙出現機率的窘境，訓練出一個雙向的語言代表模型。這使得 BERT 輸出的每個 token 的 repr. Tn 都同時蘊含了前後文資訊，真正的雙向 representation。

跟以往模型相比，BERT 能更好地處理自然語言，在著名的問答任務 SQuAD2.0 也有卓越表現

BERT 是一個強大的語言代表模型，給它一段文本序列，它能回傳一段相同長度且蘊含上下文資訊的 word repr. 序列，對下游的 NLP 任務很有幫助。

有了這樣的概念以後，我們接下來要做的事情很簡單，就是將自己感興趣的 NLP 任務的文本丟入 BERT ，為文本裡頭的每個 token 取得有語境的 word repr.，並以此 repr. 進一步 fine tune 當前任務，取得更好的結果。